In [22]:
!wandb login

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: qftie (use `wandb login --relogin` to force relogin)


In [23]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='6'

In [24]:
# Inside my model training code
import wandb

wandb.init(project="meld-emo-cls",entity='qftie',group='bert')

config = wandb.config          # Initialize config
config.batch_size = 8          # input batch size for training (default: 64)
config.test_batch_size = 64    # input batch size for testing (default: 1000)
config.epochs = 6             # number of epochs to train (default: 10)
config.lr = 2e-5               # learning rate (default: 0.01)
config.momentum = 0.1          # SGD momentum (default: 0.5) 
config.no_cuda = False         # disables CUDA training
config.bert_path = 'roberta-base'
config.exam_name = 'roberta-forwardUt'
config.max_seq_len = 400
config.max_turn = 0
config.seed = 2022


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2022-04-25 09:50:57.799716: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-25 09:50:57.799750: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [25]:
import pandas as pd 
import numpy as np 
import json, time
from tqdm import tqdm 
from sklearn.metrics import accuracy_score, classification_report, f1_score
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertModel, BertConfig, AutoTokenizer, AdamW, get_cosine_schedule_with_warmup, AutoModel, AutoConfig
import warnings
warnings.filterwarnings('ignore')

bert_path = config.bert_path
tokenizer = AutoTokenizer.from_pretrained(bert_path, truncation_side="left")   # 初始化分词器

# seed everything

In [26]:
import numpy as np
import random

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(config.seed)

# 预处理数据集

In [27]:
emo2id = {'neutral': 0, 'surprise': 1, 'fear': 2, 'sadness': 3, 'joy': 4, 'disgust': 5, 'anger': 6}

## 准备forward utterance作为sentence

In [28]:
# back_stride = int(data_text_valid['Utterance_ID'][50][-3:])
# ' '.join(data_text_valid['Utterance'][ 50- back_stride: 50].tolist())

In [29]:
# print(x_valid_forward_utt)

## tokenize

In [30]:
def load_meld_and_tokenize(file_path, train=False, context_turn = config.max_turn):
    data_text = pd.read_csv(file_path)
    data_text['Speaker_Utterance'] = data_text['Speaker'] + ':' + data_text['Utterance']
    x= (data_text['Speaker_Utterance']).tolist()
    label = [emo2id[x] for x in data_text['Emotion']]
    context = []
    for i in range(len(data_text)):
        if data_text['Utterance_ID'][i]<context_turn:
            back_stride = data_text['Utterance_ID'][i]
        else:
            back_stride = context_turn
        utt_forward = ' '.join(data_text['Speaker_Utterance'][ i- back_stride: i].tolist())
        context.append(utt_forward)
    for i in range(len(x)):
        x[i] = context[i] + '[SEP]' + x[i]
    return tokenizer(x, truncation=True, padding=True, max_length=512),label
    
train_encoding, train_label = load_meld_and_tokenize(file_path='/148Dataset/data-tie.qianfeng/MELD/data/MELD/train_sent_emo.csv')
valid_encoding, valid_label = load_meld_and_tokenize(file_path='/148Dataset/data-tie.qianfeng/MELD/data/MELD/dev_sent_emo.csv')
test_encoding, test_label = load_meld_and_tokenize(file_path='/148Dataset/data-tie.qianfeng/MELD/data/MELD/test_sent_emo.csv')

In [31]:
train_encoding['input_ids'][0]

[0,
 10975,
 3388,
 510,
 742,
 4771,
 463,
 1371,
 35,
 19726,
 38,
 21,
 5,
 477,
 621,
 15,
 127,
 138,
 17,
 27,
 29,
 3868,
 31,
 5,
 26544,
 12,
 245,
 7,
 8837,
 12,
 401,
 467,
 4,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

# 准备训练集，验证集，测试集

In [32]:
# 数据集读取 转成dict形式
class NewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    # 读取单个样本
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(int(self.labels[idx]))
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encoding, train_label)
valid_dataset = NewsDataset(valid_encoding, valid_label)
test_dataset = NewsDataset(test_encoding, test_label)


# 加载到torch的dataloader

In [33]:
# 单个读取到批量读取
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)

# train_loader = DataLoader(train_dataset, batch_size=config.batch_size, sampler=sampler)
valid_loader = DataLoader(valid_dataset, batch_size=config.batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=config.test_batch_size, shuffle=True)



# 定义bert模型

In [34]:

# 定义model
class Bert_Model(nn.Module):
    def __init__(self, bert_path, classes=7):
        super(Bert_Model, self).__init__()
        self.config = AutoConfig.from_pretrained(bert_path)  # 导入模型超参数
        self.bert = AutoModel.from_pretrained(bert_path)     # 加载预训练模型权重
        self.fc = nn.Linear(self.config.hidden_size, classes)  # 直接分类
        self.dense = nn.Linear(self.config.hidden_size, self.config.hidden_size)
        self.activation = nn.Tanh()
        self.pred = nn.Sequential(
            nn.Linear(self.config.hidden_size, 64),
            nn.Dropout(0.3),
            nn.Linear(64, classes)
        )


        
        
    def forward(self, input_ids, attention_mask=None):
        outputs = self.bert(input_ids, attention_mask, output_hidden_states= True, return_dict=True)
        # out_pool = outputs[1]   # 池化后的输出 [bs, config.hidden_size]
        out_pool = torch.mean(outputs.last_hidden_state, 1)
        # out_pool = self.dense(out_pool)
        # out_pool = self.activation(out_pool)
        logit = self.pred(out_pool)   #  [bs, classes]
        return logit

# 实例化bert模型

In [35]:
def get_parameter_number(model):
    #  打印模型参数量
    total_num = sum(p.numel() for p in model.parameters())
    trainable_num = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return 'Total parameters: {}, Trainable parameters: {}'.format(total_num, trainable_num)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = config.epochs
model = Bert_Model(bert_path)
model = nn.DataParallel(model)
model = model.cuda()
print(get_parameter_number(model))


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total parameters: 125291278, Trainable parameters: 125291278


# 优化器定义

In [36]:
optimizer = AdamW(model.parameters(), lr=config.lr, weight_decay=1e-4) #AdamW优化器
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=0.3*len(train_loader),
                                            num_training_steps=EPOCHS*len(train_loader))
# 学习率先线性warmup一个epoch，然后cosine式下降。
# 这里给个小提示，一定要加warmup（学习率从0慢慢升上去），如果把warmup去掉，可能收敛不了。

In [37]:
# # 准备放入多卡环境
# model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader)

# 定义训练函数和验证测试函数

In [38]:
# 评估模型性能，在验证集上
def evaluate(model, data_loader, device):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    val_true, val_pred = [], []
    valid_loss_sum = 0.0
    with torch.no_grad():
        for idx, batch in enumerate(data_loader):
            y_pred = model(batch['input_ids'].to(device), batch['attention_mask'].to(device))
            loss = criterion(y_pred, batch['labels'].to(device))
            y_pred = torch.argmax(y_pred, dim=1).detach().cpu().numpy().tolist()
            val_pred.extend(y_pred)
            val_true.extend(batch['labels'].cpu().numpy().tolist())
            valid_loss_sum += loss.item()
            
    print(classification_report(val_true, val_pred, digits=4))
    return accuracy_score(val_true, val_pred), valid_loss_sum/len(data_loader), f1_score(val_true, val_pred, average='macro')  #返回accuracy, loss, f1-macro


# 测试集没有标签，需要预测提交
def predict(model, data_loader, device):
    model.eval()
    val_pred = []
    with torch.no_grad():
        for batch in data_loader:
            y_pred = model(batch['input_ids'].to(device), batch['attention_mask'].to(device))
            y_pred = torch.argmax(y_pred, dim=1).detach().cpu().numpy().tolist()
            val_pred.extend(y_pred)
    return val_pred


def train_and_eval(model, train_loader, valid_loader, 
                   optimizer, scheduler, device, epoch):
    best_acc = 0.0
    patience = 0
    best_loss = 100
    best_macro_f1 = 0
    b = 0.6
    criterion = nn.CrossEntropyLoss()
    for i in range(epoch):
        """训练模型"""
        start = time.time()
        model.train()
        print("***** Running training epoch {} *****".format(i+1))
        train_loss_sum = 0.0
        for idx, batch in enumerate(train_loader):
            ids = batch['input_ids'].to(device)
            att = batch['attention_mask'].to(device)
            y = batch['labels'].to(device)  
            y_pred = model(ids, att)
            loss = criterion(y_pred, y)
            loss = (loss - b).abs() + b # This is it!
            step_lr = np.array([param_group["lr"] for param_group in optimizer.param_groups]).mean()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()   # 学习率变化
            
            train_loss_sum += loss.item()
            if (idx + 1) % (len(train_loader)//20) == 0:    # 只打印五次结果
                wandb.log({
                            'Epoch': i+1, 
                            'train_loss': loss,
                            'lr': step_lr
                            })
                print("Epoch {:04d} | Step {:04d}/{:04d} | Loss {:.4f} | Time {:.4f}".format(
                          i+1, idx+1, len(train_loader), train_loss_sum/(idx+1), time.time() - start))
                # print("Learning rate = {}".format(optimizer.state_dict()['param_groups'][0]['lr']))

        """验证模型"""
        model.eval()
        acc, valid_loss, valid_macro_f1 = evaluate(model, valid_loader, device)  # 验证模型的性能
        wandb.log({'valid_acc': acc, 'valid_loss': valid_loss})
        # 保存最优模型
        if valid_loss < best_loss:
            best_loss = valid_loss
            

        if valid_macro_f1 > best_macro_f1:
            best_macro_f1 = valid_macro_f1
            torch.save(model.state_dict(), 'pytorch_model.bin') 
            
        
        print("current macro_f1 is {:.4f}, best macro_f1 is {:.4f}".format(valid_macro_f1, best_macro_f1))
        print("time costed = {}s \n".format(round(time.time() - start, 5)))

# 训练和验证模型

In [39]:
# 训练和验证评估
train_and_eval(model, train_loader, valid_loader, optimizer, scheduler, DEVICE, EPOCHS)


***** Running training epoch 1 *****
Epoch 0001 | Step 0062/1249 | Loss 1.7949 | Time 4.6365
Epoch 0001 | Step 0124/1249 | Loss 1.6651 | Time 9.3065
Epoch 0001 | Step 0186/1249 | Loss 1.5853 | Time 13.9354
Epoch 0001 | Step 0248/1249 | Loss 1.5268 | Time 18.6432
Epoch 0001 | Step 0310/1249 | Loss 1.4769 | Time 23.3055
Epoch 0001 | Step 0372/1249 | Loss 1.4559 | Time 27.9889
Epoch 0001 | Step 0434/1249 | Loss 1.4359 | Time 32.6929
Epoch 0001 | Step 0496/1249 | Loss 1.4100 | Time 37.4027
Epoch 0001 | Step 0558/1249 | Loss 1.3935 | Time 42.4736
Epoch 0001 | Step 0620/1249 | Loss 1.3685 | Time 47.8519
Epoch 0001 | Step 0682/1249 | Loss 1.3482 | Time 52.9233
Epoch 0001 | Step 0744/1249 | Loss 1.3350 | Time 57.8328
Epoch 0001 | Step 0806/1249 | Loss 1.3334 | Time 62.6329
Epoch 0001 | Step 0868/1249 | Loss 1.3240 | Time 67.3780
Epoch 0001 | Step 0930/1249 | Loss 1.3159 | Time 72.1117
Epoch 0001 | Step 0992/1249 | Loss 1.3089 | Time 76.7968
Epoch 0001 | Step 1054/1249 | Loss 1.2983 | Time 81.6

# 加载最优模型测试

In [40]:
# 加载最优权重对测试集测试
model.load_state_dict(torch.load("pytorch_model.bin"))
pred_test = evaluate(model, test_loader, DEVICE)
# print("\n Test Accuracy = {} \n".format(accuracy_score(test_label, pred_test)))
# print(classification_report(test_label, pred_test, digits=4))
wandb.finish()

              precision    recall  f1-score   support

           0     0.7341    0.8639    0.7937      1256
           1     0.5213    0.5658    0.5427       281
           2     0.2632    0.2000    0.2273        50
           3     0.4907    0.2548    0.3354       208
           4     0.5537    0.5771    0.5652       402
           5     0.3636    0.1176    0.1778        68
           6     0.5417    0.3768    0.4444       345

    accuracy                         0.6425      2610
   macro avg     0.4955    0.4223    0.4409      2610
weighted avg     0.6199    0.6425    0.6219      2610




Epoch,▁▁▁▁▁▁▁▂▂▂▂▂▂▂▄▄▄▄▄▄▅▅▅▅▅▅▅▇▇▇▇▇▇███████
lr,▂▆███████▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train_loss,█▂▄▁▂▄▅▁▂▅▆█▅▄▁▁▃▃▃▃▃▃█▂▃▂▃▁▂▁▂▂▁▂▁▂▃▃▁▂
valid_acc,▄▆█▃▁▄
valid_loss,█▄▁▄▃▂
Epoch,6
lr,0.0
train_loss,0.81535
valid_acc,0.60595
valid_loss,1.11797


In [41]:
# output_dir = 'output/ch-roberta-dorwardUt'
# os.makedirs(output_dir, exist_ok=True)
# torch.save(model.state_dict(), output_dir+"/pytorch_model.bin")
# # torch.save(model, "pytorch_model_whole.bin")

In [42]:
torch.cuda.empty_cache()
torch.cuda.empty_cache()
torch.cuda.empty_cache()